- id
- user/id
- text
- created_at
- lang
- source
- coordinates (stored as a PostGIS point)

In [1]:
import zipfile
import json
from dateutil.parser import parse
import numpy as np
import glob

In [25]:
twitter_filenames = glob.glob('*.zip')
twitter_filenames=twitter_filenames[6:8]
twitter_filenames_json=[i.split('.')[0]+'.json' for i in twitter_filenames]

In [26]:
twitter_filenames

['westcoast-20151007-074137.json.zip', 'westcoast-20151008-091012.json.zip']

In [ ]:
rawdata=[]
i=0
for twitter_file in twitter_filenames:
    with zipfile.ZipFile(twitter_file) as z:
        with z.open(twitter_filenames_json[i]) as f:
            for line in f:
                rawdata.append(line)
    i=i+1

In [ ]:
rawdict=[]
for i in range(len(rawdata)):
    line=json.loads(rawdata[i])
    rawdict.append(line)
#     rawdict[i]['users'] = rawdict[i].pop('user')

In [ ]:
sqltw=[None]*len(rawdict)
i=0

In [ ]:
for result in rawdict: 
    try:
        if result['id']:
            ids=result['id']
            userid = result["user"]["id"]
            text = result["text"]
            lang=result['lang']
            source=result['source']
            text = text.encode('ascii', 'replace')
            time=result["created_at"]
            if result["geo"]:
                latitude = result["geo"]["coordinates"][0]
                longitude = result["geo"]["coordinates"][1]
            else:
                latitude=None
                longitude=None

            newtweet=[ids,userid,text,lang,source,latitude,longitude,time]
            sqltw[i]=newtweet
            i=i+1

    except Exception:
        continue

In [11]:
len(sqltw[2])

8

In [12]:
>>> import psycopg2
>>> conn = psycopg2.connect("dbname=siyuchen user=siyu host=localhost")
>>> cur = conn.cursor()
>>> cur.execute("""SELECT srtext FROM spatial_ref_sys WHERE srid = 4326;""")
>>> rows = cur.fetchall()
>>> print rows[0]

('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]',)


In [30]:
conn.rollback()

In [11]:
# cur.execute('DROP TABLE tweets')
# conn.commit()

In [12]:
# cur.execute("CREATE TABLE tweets(id varchar,user_id varchar,text varchar,lang varchar,source varchar,\
#                                  loc geometry(Point),time varchar);")

In [ ]:
for i in range(len(sqltw)):
    try:
        if sqltw[i]:
            if sqltw[i][6]==None:
                cur.execute("INSERT INTO tweets(id,user_id,text,lang,source,loc,time) \
                                         VALUES(%s,%s,%s,%s,%s,ST_GeomFromText(%s,%s),%s)" ,
                            (sqltw[i][0],sqltw[i][1],sqltw[i][2],sqltw[i][3],sqltw[i][4],
                             None,4326,
                             parse(sqltw[i][7])))
            else:
                cur.execute("INSERT INTO tweets(id,user_id,text,lang,source,loc,time) \
                                         VALUES(%s,%s,%s,%s,%s,ST_GeomFromText(%s,%s),%s)" ,
                            (sqltw[i][0],sqltw[i][1],sqltw[i][2],sqltw[i][3],sqltw[i][4],
                             'POINT('+str(sqltw[i][6])+' '+str(sqltw[i][5])+')',4326,
                             parse(sqltw[i][7])))
    except Exception:
        continue 
conn.commit()

In [18]:
sqltw[i]

In [19]:
i

499999